knowledge base systemm
문서를 이용한 벡터DB 구성 실습

In [1]:
!pip install docx2txt

In [2]:
# 문서 로딩 

In [69]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_table.docx')
docu = loader.load()

In [4]:
# 문서를 가지고 왔으니 쪼개기 
# 문서의 분할 -> splitter 
# large 모델은 chunk 사이즈가 커야함 
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200, # chunk size의 10% 정도
)
splitter

In [5]:
doc_list = loader.load_and_split(text_splitter=splitter)

In [6]:
len(doc_list)

183

In [7]:
# embedding 


In [8]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [10]:
# 백터 db 적재 - 크로마db, chroma / 

In [11]:
# !pip install langchain_chroma

In [12]:
from langchain_chroma import Chroma
database = Chroma.from_documents(
    documents=doc_list, embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma2'
)

In [13]:
# 불러오기 할 떄만 
# database = Chroma(collection_name='chroma-text',
#        persist_directory='./chroma', embedding_function=embedding)

In [14]:
# Retrieval


In [15]:
# 지리
query = '연봉 5천만원인 거주자의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)
retrieved_docs

[Document(id='1f32dfa0-6f34-4be7-b7d6-765253d2ffae', metadata={'source': './tax_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6%\n\n1,400만원 초과 ~ 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15%)\n\n5,000만원 초과 ~ 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24%)\n\n8,800만원 초과 ~ 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35%)\n\n1억5천만원 초과 ~ 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38%)\n\n3억원 초과 ~ 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40%)\n\n5억원 초과 ~ 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42%)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45%)\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 

In [16]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [17]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [context] 내용만 참조해서 사용자의 질문에 답변해 주세요.

[context]
{retrieved_docs}
Question : {query}
"""

In [18]:
response = llm.invoke(prompt)

In [19]:
response.content

'연봉 5천만원인 거주자의 소득세를 계산해보겠습니다.\n\n1. **과세표준**: 5천만원\n2. **적용 세율**: 5천만원 초과 ~ 8,800만원 이하의 구간에 해당합니다.\n   - 이 구간의 세율은 624만원 + (5,000만원을 초과하는 금액의 24%)입니다.\n\n3. **초과 금액 계산**: 5천만원을 초과하는 부분은 5천만원입니다.\n4. **세액 계산**:\n   - 기본 세액: 624만원\n   - 초과 금액에 대한 세액: 5,000만원 × 24% = 1,200만원\n\n5. **총 소득세**:\n   - 총 세액 = 624만원 + 1,200만원 = 1,824만원\n\n따라서 연봉 5천만원인 거주자의 소득세는 **1,824만원**입니다.'

In [20]:
# tax_table

# 질의의 정규화

In [27]:
# 질의의 품질을 높이기 위한 정규화 과정 추가
def normalize_query(q: str) -> str:
    # 숫자: '5천만원' → '5,000만원'
    q = q.replace("오천만원","5,000만원").replace("5천만원","5,000만원")
    # 용어: 연봉 → (과세) 과세표준 후보어 추가
    q = q.replace("연봉", "과세표준")
    # 용어: 직장인 -> 거주자 
    q = q.replace("직장인", "거주자")
    # 의도 신호: 계산 키워드 보강
    if "계산" not in q:
        q = q.rstrip("요?").rstrip("?") + " 계산 기준과 누진공제를 적용해 계산"
    return q

In [28]:
query = '연봉 5천만원인 직장인 소득세는 얼마인가요?'
query_n = normalize_query(query)
query_n

'과세표준 5,000만원인 거주자 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산'

In [ ]:
# 함수 이용 
retrieved_docs = database.similarity_search(query_n, k=10)
retrieved_docs

[Document(id='1f32dfa0-6f34-4be7-b7d6-765253d2ffae', metadata={'source': './tax_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6%\n\n1,400만원 초과 ~ 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15%)\n\n5,000만원 초과 ~ 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24%)\n\n8,800만원 초과 ~ 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35%)\n\n1억5천만원 초과 ~ 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38%)\n\n3억원 초과 ~ 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40%)\n\n5억원 초과 ~ 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42%)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45%)\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 

In [25]:
# query 정규화, 고도화 작업이 필요한 이유 

In [30]:
query, query_n

('연봉 5천만원인 직장인 소득세는 얼마인가요?', '과세표준 5,000만원인 거주자 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산')

In [ ]:
# retrieval RetrievalQA chain

# llm prompt 넣어서 호출했었는데
# 객체를 만들기 
retriever = database.as_retriever(
    search_type='similarity',
    search_kwargs = {'k':5}
    ) # 기본 리트리버
retriever.invoke(query_n)

[Document(id='1f32dfa0-6f34-4be7-b7d6-765253d2ffae', metadata={'source': './tax_table.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n\n\n종합소득 과세표준\n\n세율\n\n1,400만원 이하\n\n과세표준의 6%\n\n1,400만원 초과 ~ 5,000만원 이하\n\n84만원 + (1,400만원을 초과하는 금액의 15%)\n\n5,000만원 초과 ~ 8,800만원 이하\n\n624만원 + (5,000만원을 초과하는 금액의 24%)\n\n8,800만원 초과 ~ 1억5천만원 이하\n\n1,536만원 + (8,800만원을 초과하는 금액의 35%)\n\n1억5천만원 초과 ~ 3억원 이하\n\n3,706만원 + (1억5천만원을 초과하는 금액의 38%)\n\n3억원 초과 ~ 5억원 이하\n\n9,406만원 + (3억원을 초과하는 금액의 40%)\n\n5억원 초과 ~ 10억원 이하\n\n1억7,406만원 + (5억원을 초과하는 금액의 42%)\n\n10억원 초과\n\n3억8,406만원 + (10억원을 초과하는 금액의 45%)\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<개정 2013. 1. 1., 2014. 12. 23.>\n\n1. 해당 과세기간의 퇴직소득과세표준에 제1항의 세율을 적용하여 계산한 금액\n\n2. 제1호의 금액을 12로 나눈 금액에 근속연수를 곱한 금액\n\n3. 삭제<2014. 12. 23.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2관 세액공제 <개정 2009. 12. 

In [ ]:
from langchain import hub
# 만들어져있는 프롬프트를 pull 해오기, git에서 pull  하는 방식과 같음 
prompt = hub.pull('rlm/rag-prompt') # 범용 래그 프롬프트

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [43]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [44]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs = {'prompt':prompt}
)

In [ ]:
response = qa_chain.invoke({'query':query_n}) # 소득세법 문서 추출 체인 / query를 받음
response

{'query': '과세표준 5,000만원인 거주자 소득세는 얼마인가 계산 기준과 누진공제를 적용해 계산',
 'result': '과세표준이 5,000만원인 거주자의 소득세는 84만원 + (1,400만원 초과 금액의 15%)로 계산합니다. 5,000만원의 경우, 1,400만원 초과 금액은 3,600만원이므로, 소득세는 84만원 + (3,600만원 × 15%) = 84만원 + 540만원 = 624만원이 됩니다. 따라서 총 소득세는 624만원입니다.'}

In [ ]:
# Retrieval를 위한 키워드 사전 (용어사전)

In [49]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ['사람을 나타내는 표현 --> 거주자']

prompt_dic = ChatPromptTemplate.from_template(
    f'''
    사용자의 질문을 보고, 아래의 사전을 참고해서 사용자의 질문을 변경하세요.
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴하세요. 
    사전 : {dictionary}
    
    질문 : {{question}}
    '''
)

# 템플릿 완성 

In [ ]:
dictionary_chain = prompt_dic | llm | StrOutputParser()
tax_chain = {'query':dictionary_chain} | qa_chain # 처음 input은 question, -> 그다음 query로 넘겨줘야함 dictionary_chain를 쿼리로 전달해줘야함, 마지막 체인 

In [53]:
query

'연봉 5천만원인 직장인 소득세는 얼마인가요?'

In [54]:
new_query = dictionary_chain.invoke({'question':query})
new_query

'연봉 5천만원인 거주자 소득세는 얼마인가요?'

In [ ]:
tax_chain.invoke({'question':query}) 

{'query': '연봉 5천만원인 거주자 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만원에 1,400만원을 초과하는 금액의 15%를 더한 금액으로 계산됩니다. 따라서 소득세는 84만원 + (5,000만원 - 1,400만원) × 0.15로 계산할 수 있습니다. 즉, 84만원 + 510만원 × 0.15 = 84만원 + 76.5만원 = 160.5만원입니다.'}

In [65]:
query_2 = '기부자별 발급명세 미작성 금액 합계 팔천만원이면, 가산세는 얼마인가요?'

In [68]:
# 질의의 품질을 높이기 위한 정규화 과정 추가
def normalize_query(q: str) -> str:
    # 숫자: '5천만원' → '5,000만원'
    q = q.replace("8천만원","80,000,000원").replace("팔천만원","80,000,000원")
    return q
query_2n = normalize_query(query_2)
query_2n
retrieved_docs = database.similarity_search(query_2, k=5)
retrieved_docs

[Document(id='60768705-c3c4-4a75-9b42-209c6c746c7d', metadata={'source': './tax_table.docx'}, page_content='[본조신설 2019. 12. 31.]\n\n\n\n제81조의6(증명서류 수취 불성실 가산세) ① 사업자(대통령령으로 정하는 소규모사업자 및 대통령령으로 정하는 바에 따라 소득금액이 추계되는 자는 제외한다)가 사업과 관련하여 다른 사업자(법인을 포함한다)로부터 재화 또는 용역을 공급받고 제160조의2제2항 각 호의 어느 하나에 따른 증명서류를 받지 아니하거나 사실과 다른 증명서류를 받은 경우에는 그 받지 아니하거나 사실과 다르게 받은 금액으로 필요경비에 산입하는 것이 인정되는 금액(건별로 받아야 할 금액과의 차액을 말한다)의 100분의 2를 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다. 다만, 제160조의2제2항 각 호 외의 부분 단서가 적용되는 부분은 그러하지 아니하다.\n\n② 제1항에 따른 가산세는 종합소득산출세액이 없는 경우에도 적용한다.\n\n[본조신설 2019. 12. 31.]\n\n\n\n제81조의7(기부금영수증 발급ㆍ작성ㆍ보관 불성실 가산세) ① 제34조 및 「법인세법」 제24조에 따라 기부금을 필요경비 또는 손금에 산입하거나, 제59조의4제4항에 따라 기부금세액공제를 받기 위하여 필요한 기부금영수증[「법인세법」 제75조의4제2항에 따른 전자기부금영수증(이하 “전자기부금영수증”이라 한다)을 포함한다. 이하 “기부금영수증”이라 한다]을 발급하는 거주자 또는 비거주자가 다음 각 호의 어느 하나에 해당하는 경우에는 다음 각 호의 구분에 따른 금액을 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다. <개정 2020. 12. 29.>\n\n1. 기부금영수증을 사실과 다르게 적어 발급(기부금액 또는 기부자의 인적사항 등 주요 사항을 적지 아니하고 발급하는 경우를 포함한다. 이하 이 호에서 같다)한 경우\n\n가. 기부금액을 사

In [58]:
from langchain import hub
prompt = hub.pull('rlm/rag-prompt') # 범용 래그 프롬프트

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [59]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
# retriever: 예상 답변이자 나와야하는 답변 
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs = {'prompt':prompt}
)

In [61]:
response = qa_chain.invoke({'query':query_2}) # 소득세법 문서 추출 체인 / query를 받음
response

{'query': '기부자별 발급명세 미작성 금액 합계 80,000,000원이면, 가산세는 얼마인가요?',
 'result': '기부자별 발급명세 미작성 금액 합계가 80,000,000원일 경우, 가산세는 80,000,000원의 0.2%인 160,000원이 됩니다. 따라서, 가산세는 160,000원입니다.'}

In [57]:
retriever = database.as_retriever(
    search_type='similarity',
    search_kwargs = {'k':5}
    ) # 기본 리트리버
retriever.invoke(query_2)

[Document(id='60768705-c3c4-4a75-9b42-209c6c746c7d', metadata={'source': './tax_table.docx'}, page_content='[본조신설 2019. 12. 31.]\n\n\n\n제81조의6(증명서류 수취 불성실 가산세) ① 사업자(대통령령으로 정하는 소규모사업자 및 대통령령으로 정하는 바에 따라 소득금액이 추계되는 자는 제외한다)가 사업과 관련하여 다른 사업자(법인을 포함한다)로부터 재화 또는 용역을 공급받고 제160조의2제2항 각 호의 어느 하나에 따른 증명서류를 받지 아니하거나 사실과 다른 증명서류를 받은 경우에는 그 받지 아니하거나 사실과 다르게 받은 금액으로 필요경비에 산입하는 것이 인정되는 금액(건별로 받아야 할 금액과의 차액을 말한다)의 100분의 2를 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다. 다만, 제160조의2제2항 각 호 외의 부분 단서가 적용되는 부분은 그러하지 아니하다.\n\n② 제1항에 따른 가산세는 종합소득산출세액이 없는 경우에도 적용한다.\n\n[본조신설 2019. 12. 31.]\n\n\n\n제81조의7(기부금영수증 발급ㆍ작성ㆍ보관 불성실 가산세) ① 제34조 및 「법인세법」 제24조에 따라 기부금을 필요경비 또는 손금에 산입하거나, 제59조의4제4항에 따라 기부금세액공제를 받기 위하여 필요한 기부금영수증[「법인세법」 제75조의4제2항에 따른 전자기부금영수증(이하 “전자기부금영수증”이라 한다)을 포함한다. 이하 “기부금영수증”이라 한다]을 발급하는 거주자 또는 비거주자가 다음 각 호의 어느 하나에 해당하는 경우에는 다음 각 호의 구분에 따른 금액을 가산세로 해당 과세기간의 종합소득 결정세액에 더하여 납부하여야 한다. <개정 2020. 12. 29.>\n\n1. 기부금영수증을 사실과 다르게 적어 발급(기부금액 또는 기부자의 인적사항 등 주요 사항을 적지 아니하고 발급하는 경우를 포함한다. 이하 이 호에서 같다)한 경우\n\n가. 기부금액을 사

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ['사람을 나타내는 표현 --> 거주자']

prompt_dic = ChatPromptTemplate.from_template(
    f'''
    사용자의 질문을 보고, 아래의 사전을 참고해서 사용자의 질문을 변경하세요.
    만약 변경할 필요가 없다고 판단되면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴하세요. 
    사전 : {dictionary}
    
    질문 : {{question}}
    '''
)

# 템플릿 완성 

In [ ]:
dictionary_chain = prompt_dic | llm | StrOutputParser()
tax_chain = {'query':dictionary_chain} | qa_chain # 처음 input은 question, -> 그다음 query로 넘겨줘야함 dictionary_chain를 쿼리로 전달해줘야함, 마지막 체인 

In [ ]:
query

'연봉 5천만원인 직장인 소득세는 얼마인가요?'

In [ ]:
new_query = dictionary_chain.invoke({'question':query})
new_query

'연봉 5천만원인 거주자 소득세는 얼마인가요?'

In [ ]:
tax_chain.invoke({'question':query}) 

{'query': '연봉 5천만원인 거주자 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만원에 1,400만원을 초과하는 금액의 15%를 더한 금액으로 계산됩니다. 따라서 소득세는 84만원 + (5,000만원 - 1,400만원) × 0.15로 계산할 수 있습니다. 즉, 84만원 + 510만원 × 0.15 = 84만원 + 76.5만원 = 160.5만원입니다.'}